### Yield-Curves, forwards and swaps
All market instruments are constraints on a single arbitrage-free ZCB curve. Once we have ZCB prices we can construct forward rates, spot rates and swap rates. 
A zero-coupon bond price p(0,T) tells us: How much 1 unit of money paid at time T is worth today. 
Therefore the ZCB Curve is like the exchange rate between today and the future. 

1. **Spot rates**  
   What is the average discount from 0 to T. 6M EURIBOR fixing: rate of a loan from 0 to 0.5 years -> pins down the ZCB price p(0,0.5).

2. **Forward rates**  
  - Discount rate between two future dates
  - FRAs don’t give a single ZCB price → they link two ZCB points via a ratio


3. **Swap rates**  
  - Fixed rate that makes PV(fixed) = PV(floating)
  - Floating leg = sum of forward rates
  - Swap rate ≈ weighted average of forwards


#### 1.a: Fitting a yield curve of continuously compounded ZCB rates to the market data.
1. Start by writing up the given data in a structure as shown below.

In [ ]:
# DATA FOR YIELD CURVE CONSTRUCTION
Market_prices = np.array([x,x1,x2])
EURIBOR_fixing = [{"id": 0,"instrument": "libor","maturity": 1/2, "rate":0.03772}]
fra_market = [{"id": 1,"instrument": "fra","exercise": 1/12,"maturity": 7/12, "rate": 0.04026}]
## FRA = 1X7 means exercise = 1/12 and maturity = 7/12
swap_market = [{"id": 10,"instrument": "swap","maturity": 2, "rate": 0.05228, "float_freq": "semiannual", "fixed_freq": "annual","indices": []}]
## IRS = Interest rate swap IRS = 2Y -> Maturity 2 years. 
data_zcb = EURIBOR_fixing + fra_market + swap_market

2. Then choose which fit/type of interpolation you see most fit. 

In [ ]:
mesh = 1/12          # here is chosen a monthly mesh 
M = 360              # 30 years * 12
# interpolation options for the yield curve you can choose between linear, cubic spline and hermite.
interpolation_options = {"method": "hermite", "degree": 3, "transition": "smooth"} 
interpolation_options = {"method":"linear","transition": "smooth"}
interpolation_options = {"method":"nelson_siegel","transition": "smooth"}

# FITTING THE YIELD CURVE, T_fit produces knot points in years, R_fit the corresponding spot rates
T_fit, R_fit = fid.zcb_curve_fit(data_zcb, interpolation_options=interpolation_options)

# PLOTTING THE YIELD CURVE
T_inter = np.array([i*mesh for i in range(0,M+1)])
p_inter, R_inter, f_inter, T_inter = fid.zcb_curve_interpolate(T_inter,T_fit,R_fit,interpolation_options = interpolation_options)
#T_inter: Interpolated time points in years
#p_inter: Interpolated discount factors (zero-coupon prices)
#R_inter: Interpolated spot rates
#f_inter: Interpolated forward rates

3. Reporting the 6M, 1Y, 2Y, 5Y, 10Y, 15Y, 20Y and 30Y continuously compounded spot rates.
Given the I have fitted a continuous spot rate curve we computed above we now read off the spot rates at the maturities the exam asks for using the function below.

In [1]:
# Function fid.for_values_in_list_find_value_return_value looks up for each maturity in the first list the corresponding spot rates from the interpolated curve
R_output = fid.for_values_in_list_find_value_return_value([0.5,1,2,5,10,15,20,30],T_inter,R_inter)
# r0 = short rate at time 0
r0 = R_inter[0]
print(f"Problem 1a - 6M,1Y,2Y,5Y,10Y,15Y,20Y,30Y spot rates from the fit: {np.round(R_output,5)}")

NameError: name 'fid' is not defined

SSE: The SSE from the swap portion is of order $10^{-25}$ and from the FRA portion of order $10^{-10}$


#### 1.b: Discuss properties of spot and forward rates + curve type
- Spot rates should be continuous in maturity (similar cash flows should be discounted similarly).
- Forward rates should be well-behaved (preferably positive and smooth) to avoid unrealistic derivative prices.
- A smooth yield curve fit is chosen such that spot rates are continuous and forward rates are well-behaved. LOOK AT YOUR PLOT and see how the fitted curve fits these proporties and if it provides and arbitrage-free representation of the market term structure. 

#### 1.c: Computing 6M forward Euribor rates up to $T=10$ years and compute the 10Y par swap rate.
First we find discount factors at each semiannual date to compute the forward Euribor and the present values of the par swap. 

fid.forward_libor_rates_from_zcb_prices:
$$ L(0;T_{i-1},T_i)=\frac{p(0,T_{i-1}-p(0,T_i))}{\alpha p(0,T_i)}$$

fid.swap_rate_from_zcb_prices (par fixed rate at which swap value 0 at inception):
$$ R_{swap}=\frac{1-p(0,T_N)}{\sum^N_{i=1} \Delta_i p(0,T_i)}$$

Where $S=\sum \Delta_i p(0,T_i)$ is the accrual factor.



In [ ]:
#alpha_floating_leg means semiannual payments i.e. EURIBOR 6M
alpha_floating_leg = 0.5
#T_10Y_swap is the vector of payment dates for a 10Y swap with semiannual floating leg payments
#As we include period 0, we have 21 payments.
T_10Y_swap = np.array([i*alpha_floating_leg for i in range(0,21)])
#fid.function looks for each maturity in the first list the corresponding ZCB prices. 
#Finds: discount factors: p(0,0),p(0,0.5),p(0,1),...,p(0,10) needed to price cash flows on those dates.
p_10Y_swap = fid.for_values_in_list_find_value_return_value(T_10Y_swap,T_inter,p_inter)
#fid.forward function finds the forward Euribor for each 6M period. 
L_6M = fid.forward_libor_rates_from_zcb_prices(T_10Y_swap,p_10Y_swap,horizon = 1)
#Now we can compute the par swap rate for the 10Y swap
R_10Y_swap, S_10Y_swap = fid.swap_rate_from_zcb_prices(0,0,10,"annual",T_10Y_swap,p_10Y_swap)
print(f"Problem 1c - 10Y par swap rate: {R_10Y_swap}. The par swap rate is the fixed rate that exactly compensates for the floating leg in PV terms.")